# 导入所需模块

In [ ]:
import os
import shutil
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

# 加载数据集
加载之前收集好的数据

In [ ]:
data_dir = 'datasets'

# 删除缓存文件夹：'.ipynb_checkpoints'
dir_list = os.listdir(data_dir)
if '.ipynb_checkpoints' in dir_list:
    shutil.rmtree(os.path.join(data_dir,'.ipynb_checkpoints'))

dataset = datasets.ImageFolder(
    data_dir,
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)
class_idx = dataset.class_to_idx
print('数据标签与对应的索引',class_idx)

# 划分训练集与测试集

In [ ]:
# 划分训练集和测试集
valid_percente = 0.2
num_valid = int(len(dataset)*valid_percente)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_valid, num_valid])
num_train_dataset = len(train_dataset)
num_test_dataset = len(test_dataset)

# 将数据放入加载器

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

# 加载网络结构与预训练模型参数

In [ ]:
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, len(class_idx))

# 将模型转移到GPU

In [ ]:
device = torch.device('cuda')
model = model.to(device)

# 开始训练

In [ ]:
NUM_EPOCHS = 10

model_path = r'students_model/rps.pth'

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

loss_fc = torch.nn.CrossEntropyLoss()

def train():
    
    for epoch in range(NUM_EPOCHS):
        # 初始化训练参数
        best_accuracy = 0.0
        train_loss = 0.0
        train_corrects = 0

        # 初始化测试参数
        test_acc = 0.0
        test_loss = 0.0
        test_corrects = 0
        # 模型训练
        model.train()

        for images, labels in iter(train_loader):
            # 选择设备将“图片”和“标签”输入模型中
            images = images.to(device)
            labels = labels.to(device)
            # 初始化梯度
            optimizer.zero_grad()
            # 模型前向传播
            outputs = model(images)
            # 通过交叉熵求出模型预测的结果与真实“标签”之间的误差值loss
            tr_loss = loss_fc(outputs, labels)
            # 反向传播，通过loss对模型参数进行求导更新参数
            tr_loss.backward()
            # 使用优化器对模型参数进行更新
            optimizer.step()

            train_loss += tr_loss.item() * images.size(0)

            _, predict = torch.max(outputs, 1)
            train_corrects += torch.sum(labels.data == predict)

        train_loss = train_loss / num_train_dataset
        train_acc = train_corrects.item() / num_train_dataset
        # 对测试集进行评估
        model.eval()

        for images, labels in iter(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            with torch.no_grad():
                # 前向传播得到预测结果
                outputs = model(images)
                _, predict = torch.max(outputs, 1)
                t_loss = loss_fc(outputs, labels)
                test_loss += t_loss.item() * images.size(0)

                # 记录预测正确的数量
                test_corrects += torch.sum(labels.data == predict)

        test_loss = test_loss / num_test_dataset
        test_acc = test_corrects.item() / num_test_dataset

        print('epoch={}'.format(epoch + 1))
        print('训练数据集准确率为：{:.2%}，误差为：{:.5f}'.format(train_acc, train_loss))
        print('测试数据集准确率为：{:.2%}, 误差为：{:.5f}'.format(test_acc, test_loss))
        if test_acc > 0.6:
            if test_acc > best_accuracy:
                torch.save(model, model_path)
                best_accuracy = test_acc

In [ ]:
train()